In [13]:
import numpy as np
import matplotlib.pyplot as plt
import time
import imageio
import os

In [15]:
nodos = 100
L = 1
k = 1
w = 1
theta_B = 40

array([0.3       , 0.29999916, 0.29999915, 0.29999914, 0.29999913,
       0.29999912, 0.29999912, 0.29999911, 0.2999991 , 0.29999909,
       0.29999907, 0.29999906, 0.29999905, 0.29999904, 0.29999903,
       0.29999902, 0.299999  , 0.29999899, 0.29999775, 0.29999511,
       0.2999925 , 0.29998992, 0.29998737, 0.29998484, 0.29998234,
       0.29997986, 0.2999774 , 0.29997496, 0.29997254, 0.29997014,
       0.29996775, 0.29996537, 0.29996301, 0.29996065, 0.2999583 ,
       0.29995596, 0.29995362, 0.29995128, 0.29994729, 0.2999426 ,
       0.29993796, 0.29993337, 0.29992881, 0.2999243 , 0.29991981,
       0.29991536, 0.29991094, 0.29990654, 0.29990217, 0.29989781,
       0.29989347, 0.29988914, 0.29988483, 0.29988052, 0.29987495,
       0.29986939, 0.29986222, 0.2982843 , 0.29444503, 0.28886769,
       0.2817018 , 0.27401116, 0.26642293, 0.25892486, 0.25147361,
       0.24406592, 0.23670255, 0.2293827 , 0.2221053 , 0.21486924,
       0.20767334, 0.20051628, 0.19339664, 0.18631283, 0.17926

In [3]:
def eficiencia(perfil,params):
    
    L,k,h,B0,B1,w,theta_B = params
    
    perfil_inferior = B1*(1-np.linspace(0,1,nodos))
    P = 2*(perfil+perfil_inferior+w)
    A = (perfil_inferior+perfil)*w
    dx = L/nodos

    coeficientes = np.zeros((nodos,nodos))
    coeficientes[0,0] = 1 #Theta(0) = Theta_B
    
    coeficientes[-1,-1] = 3
    coeficientes[-1,-2] = -4
    coeficientes[-1,-3] = 1 #theta_i-2 - 4theta_i-1 + 3theta_i = 2theta_prima*deltaX

    constantes = np.zeros(nodos)
    constantes[0] = theta_B

    for i in range(1,nodos-1):

        coeficientes[i,i] = -2*A[i] - (h/k)*P[i]*dx**2
        coeficientes[i,i-1] = A[i]-(A[i+1]-A[i-1])/4
        coeficientes[i,i+1] = A[i]+(A[i+1]-A[i-1])/4

    A_inv = np.linalg.inv(coeficientes)
    Thetas = np.matmul(A_inv,constantes)
    
    Q = np.trapz(h*P*Thetas,np.linspace(0,L,nodos))
    Qm = np.trapz(h*P*theta_B,np.linspace(0,L,nodos))
    Eta = Q/Qm
    return Eta, Thetas

In [4]:
h = 0.2
B0 = 0.3
B1 = 0.3
perfil_tri = B0-np.linspace(0,B0,nodos)
params = L,k,h,B0,B1,w,theta_B
etaTri,thetasTri = eficiencia(perfil_tri,params)

h = 0.2
B0 = 0.3
B1 = 0
perfil_rect = B0*np.ones(nodos)
params = L,k,h,B0,B1,w,theta_B
etaRect,thetasRect = eficiencia(perfil_rect,params)

In [5]:
print('--------------------------------------------------------------')
print('--------------------------------------------------------------')
print()
print('Eficiencia aleta triangular = {:.3f}'.format(etaTri))
print('Eficiencia aleta rectangular = {:.3f}'.format(etaRect))
print()
print('--------------------------------------------------------------')
print('--------------------------------------------------------------')


# plt.figure()
# plt.plot(np.linspace(0,L,nodos),thetasRect)
# plt.close()

plt.figure()
plt.plot(np.linspace(0,L,nodos),thetasTri)
plt.close()

--------------------------------------------------------------
--------------------------------------------------------------

Eficiencia aleta triangular = 0.756
Eficiencia aleta rectangular = 0.662

--------------------------------------------------------------
--------------------------------------------------------------


In [6]:
tiempo0 = time.time()

perfil_prueba = perfil_rect

paso = 0.1
N_max = 400

cambio = []
etas = []
perfiles = []
cte_gradiente = 1/(L*np.sqrt(h/(k*B0)))
iteraciones = 0

eta = 0
eta_nuevo = 1
gradiente = np.zeros(nodos)

#while eta_nuevo > eta:
for j in range(N_max):
    
    perfil_prueba = perfil_prueba+gradiente
    iteraciones += 1
    gradiente = [0]
    eta,thetas = eficiencia(perfil_prueba,params)
    etas.append(eta)
    perfiles.append(perfil_prueba)
    for i in range(1,nodos):
        
        perfil_nuevo = perfil_prueba.copy()
        perfil_nuevo[i] += paso
        eta_i,thetas = eficiencia(perfil_nuevo,params)
        nabla_i = (eta_i-eta)
        
        if nabla_i*cte_gradiente + perfil_prueba[i] <= 0 or nabla_i*cte_gradiente+perfil_prueba[i]>B0:
            nabla_i = 0
        gradiente.append(nabla_i)

    
    gradiente = np.array(gradiente)*cte_gradiente
    norma_nabla = np.sqrt(np.sum(np.square(gradiente)))
    cambio.append(norma_nabla)
    
    eta_nuevo,thetas = eficiencia(perfil_prueba+gradiente,params)
    
    

print('Tiempo por iteración = {:.2f}'.format((time.time()-tiempo0)/iteraciones))
print('Iteraciones = ' + str(iteraciones) + '; Tiempo = ' + str(time.time()-tiempo0))

NameError: name 'paso' is not defined

In [ ]:
color1 = 'slategray'
color2 = 'black'
with imageio.get_writer('optimization_evolution.gif', mode='I') as writer:
    for i in range(N_max):
        img = plt.figure(figsize=(12,6))
        plt.plot(np.linspace(0,L,nodos),perfiles[i],color=color2)
#         plt.plot(np.linspace(0,L,nodos),B0*np.ones(nodos))
        plt.plot([0,L],[B1]*2,color=color2)
        plt.plot([L,L],[B1,perfiles[i][-1]],color=color2)
        plt.fill_between(np.linspace(0,L,nodos),perfiles[i],B1,color=color1)
        plt.title('$\eta$ = {:.2f}'.format(etas[i]))
        plt.ylim(B1-0.12,0.42)
        plt.savefig(str(i),bbox_inches='tight',transparent=False,facecolor='white')
        plt.close(img)
        image = imageio.imread(str(i)+".png")
        writer.append_data(image)
        os.remove(str(i)+".png")

# with imageio.get_writer('optimization_evolution.gif', mode='I') as writer:
#     for filename in range(N_max):
#         image = imageio.imread(str(filename)+".png")
#         writer.append_data(image)
#         os.remove(str(filename)+".png")

In [ ]:
eta,thetas = eficiencia(perfil_prueba,params)

plt.figure()
plt.plot(np.linspace(0,L,nodos),perfil_prueba)
plt.plot(np.linspace(0,L,nodos),B0*np.ones(nodos))
plt.ylim(0,B0+0.1)
plt.title('$\eta$ = {:.2f}'.format(eta))

plt.figure()
plt.plot(np.linspace(0,L,nodos),perfiles[286])
plt.plot(np.linspace(0,L,nodos),B0*np.ones(nodos))
plt.ylim(0,B0+0.1)
plt.title('$\eta$ = {:.2f}'.format(etas[286]))

plt.figure()
plt.plot(np.linspace(1,len(cambio),len(cambio)),cambio,label='Magnitud del gradiente')
#plt.plot(np.linspace(1,len(cambio),len(cambio)),etas,label = 'Eficiencia')
plt.title('Eficiencia y Gradiente en cada iteración')
plt.legend(loc=1)

In [ ]:
plt.figure()
plt.plot(np.linspace(0,L,nodos),thetas)
plt.title('Perfil de temperaturas (Perfil fallido)')

eta,thetas = eficiencia(perfiles[286],params)
plt.figure()
plt.plot(np.linspace(0,L,nodos),thetas)
plt.title('Perfil de temperaturas (Mejor perfil)')

# plt.figure()
# plt.plot(np.linspace(0,L,nodos),B0-np.linspace(0,B0,nodos),c='r')
# plt.plot(np.linspace(0,L,nodos),-0.3+np.linspace(0,0.3,nodos),c='r')

# plt.plot(np.linspace(0,L,nodos),B0*np.ones(nodos),c='b')
# plt.plot(np.linspace(0,L,nodos),0*np.ones(nodos),c='b')